# Introdução

O projeto, da unidade curricular de Inteligência Artificial da Licenciatura em Engenharia de Sistemas Informáticos (2024-25), tem como objetivo implementar abordagens e métodos distintos de Aprendizagem Automática (ML) para resolver um problema específico utilizando um conjunto de dados públicos.

O trabalho foi realizado pelos seguintes membros:

- João Ponte - 17694
- João Carvalho - 12747

Para a realização deste projeto foi selecionado o seguinte dataset público:
- https://www.kaggle.com/code/ryanholbrook/clustering-with-k-means/tutorial

Link para o repositório do projeto no Github:
- https://github.com/a12747/IA24_G11_2

# Desenvolvimento

## Instalação de livrarias

In [101]:
!pip install matplotlib
!pip install seaborn
!pip install xgboost
!pip install mlxtend

## Importação de livrarias

In [102]:
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn para ML
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Modelos de Classificação
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.tree import DecisionTreeClassifier

# Para Regras de Associação
from mlxtend.frequent_patterns import apriori, association_rules

class XGBClassifierWithTags(XGBClassifier):
    """
    A temporary wrapper around XGBClassifier to define `__sklearn_tags__`
    for scikit-learn 1.7 compatibility.
    """
    def __sklearn_tags__(self):
        """
        Return a dictionary of tags for scikit-learn's inspection.
        Adjust these tags depending on the capabilities of XGBoost:
          - 'allow_nan': whether the estimator can handle NaN values
          - 'X_types': list of input types supported ('2darray', 'sparse', etc.)
          - 'non_deterministic': True if there's randomness
          - And so on.
        """
        # Here’s a minimal set of tags:
        tags = {
            "allow_nan": True,         # XGBoost can handle missing values (as per doc)
            "X_types": ["2darray"],    # typically 2D array input
            "non_deterministic": True, # random seeds can affect results
            "poor_score": False,       # used for scikit-learn internal testing 
            "no_validation": False     # we do want normal input validation
        }
        return tags

## Carregamento de Ficheiro

In [103]:
# Carregar dados
df = pd.read_csv("./datasets/housing.csv")

# Visualizar as 5 primeiras linhas
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Classificação

### Definição de threshold

In [104]:
# Definir um valor de base das casas
threshold_value = 2.5
df['HighValue'] = (df['MedHouseVal'] > threshold_value).astype(int)

# Check distribution
df['HighValue'].value_counts()

0    14858
1     5782
Name: HighValue, dtype: int64

### Definição de features e resultados a atingir

In [105]:
# Colunas a utilizar para obter resultados
feature_cols = [
    'MedInc',
    'HouseAge',
    'AveRooms',
    'AveBedrms',
    'Population',
    'AveOccup',
    'Latitude',
    'Longitude'
]

class_names = ["LowValue", "HighValue"]

X = df[feature_cols].copy()
y = df['HighValue']

### Definição de testes do modelo

In [106]:
# Definição de treino do modelo
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [107]:
# Definir apenas uma escala de dados a utilizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)
# print(X_train_scaled)
# print(X_test_scaled)

### Lógica de regressão

In [108]:
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_scaled, y_train)
y_pred_logreg = logreg.predict(X_test_scaled)

acc_logreg = accuracy_score(y_test, y_pred_logreg)
f1_logreg  = f1_score(y_test, y_pred_logreg)

print("Logistic Regression - Accuracy:", acc_logreg)
print("Logistic Regression - F1 Score:", f1_logreg)

Logistic Regression - Accuracy: 0.8524709302325582
Logistic Regression - F1 Score: 0.7142186766776162


### Random Forest

In [109]:
rf = RandomForestClassifier(random_state=42)

param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None]
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='f1', n_jobs=-1)
grid_rf.fit(X_train_scaled, y_train)

best_rf = grid_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test_scaled)

acc_rf = accuracy_score(y_test, y_pred_rf)
f1_rf  = f1_score(y_test, y_pred_rf)

print("Random Forest - Best Params:", grid_rf.best_params_)
print("Random Forest - Accuracy:", acc_rf)
print("Random Forest - F1 Score:", f1_rf)

Random Forest - Best Params: {'max_depth': None, 'n_estimators': 100}
Random Forest - Accuracy: 0.9009205426356589
Random Forest - F1 Score: 0.812299219825608


In [110]:
# -------------------------------
# 1. Train your Decision Tree
# -------------------------------
DTC = DecisionTreeClassifier(random_state=42)
DTC.fit(X_train, y_train)   # Where X_train and y_train are your training data/labels

# -------------------------------
# 2. Predict on the test set
# -------------------------------
predicted = DTC.predict(X_test)

# -------------------------------
# 3. Confusion Matrix & Accuracy
# -------------------------------
print("Confusion Matrix")
confm = confusion_matrix(y_test, predicted, labels=class_names)
print(confm)

print("Accuracy Score : ", accuracy_score(y_test, predicted))
print("Report: ")
print(classification_report(y_test, predicted, target_names=["Class 0", "Class 1"]))

# -------------------------------
# 4. Mean Absolute Error (MAE)
# -------------------------------
# Convert the true and predicted labels to numeric (if they aren't already).
mae_y_true = list(map(convert_num, y_test))
mae_y_pred = list(map(convert_num, predicted))
print(f"Mean Absolute Error: {mean_absolute_error(mae_y_true, mae_y_pred)}")

# -------------------------------
# 5. Confusion Matrix Heatmap
# -------------------------------
# Create a DataFrame for the confusion matrix
df_cm = pd.DataFrame(confm, index=class_names, columns=class_names)

plt.figure(figsize=(6,4))  # Adjust figure size if you like
ax = sns.heatmap(df_cm, cmap="Oranges", annot=True, fmt="d")
ax.set_xlabel("Predicted", fontsize=12)
ax.set_ylabel("Actual", fontsize=12)
plt.title("Confusion Matrix")
plt.show()

Confusion Matrix


ValueError: At least one label specified must be in y_true

### XGBoost

In [86]:
xgb_model = XGBClassifierWithTags(random_state=42, 
                                  use_label_encoder=False, 
                                  eval_metric='logloss')

param_grid_xgb = {
    'n_estimators': [50, 100],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1]
}

grid_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='f1', n_jobs=-1)
grid_xgb.fit(X_train_scaled, y_train)

best_xgb = grid_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test_scaled)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
f1_xgb  = f1_score(y_test, y_pred_xgb)

print("XGB - Best Params:", grid_xgb.best_params_)
print("XGB - Accuracy:", acc_xgb)
print("XGB - F1 Score:", f1_xgb)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

# Conclusão

O desenvolvimento deste projeto permitiu aplicar conceitos fundamentais de Inteligência Artificial na criação e implementação de um agente inteligente. 
Durante este processo, foi possível explorar técnicas de planeamento, otimização e resolução de problemas sob restrições, consolidando conhecimentos teóricos e práticos.
Foi programado um agente inteligente capaz de lidar com restrições fortes, como precedência de tarefas e capacidade de recursos, e restrições leves, como a minimização da duração total do calendário (makespan).
Estas restrições foram integradas de maneira a criar um modelo realista e aplicável a cenários complexos.
Ao longo do processo, explorámos diversos tipos de soluções, desde soluções possíveis, que podem violar restrições, até soluções viáveis e otimizadas, que garantem o cumprimento das condições definidas.
O agente foi desenhado para encontrar soluções com pontuações elevadas dentro de um tempo limitado, demonstrando a capacidade de identificar a melhor solução encontrada e, quando possível, soluções ótimas.
